In [ ]:
import numpy as np

def clip_gradients(grads, max_norm):
    total_norm = 0
    
    # Compute the norm of the gradients
    grad_norm = np.linalg.norm(grads)

    # Clip the gradients if the norm is too large
    clip_threshold = 1.0
    if grad_norm > clip_threshold:
        clipped_grads = grads * (clip_threshold / grad_norm)
    else:
        clipped_grads = grads

    return grads

In [ ]:
'''
This is an two layered neural network
when initiating set the number of layers accordingly!!!!!
'''

import numpy as np
from scipy.stats import multivariate_normal

class NeuralNet_actor():
    def __init__(self,layers,lr,iterations=1000):
        self.params={}
        self.lr=lr
        self.iterations=iterations
        self.loss=[]
        self.sample_size=None
        self.layers=layers
        np.random.seed(1)
        self.params['w1']=np.random.randn(self.layers[0],self.layers[1])
        self.params['b1']=np.random.rand(self.layers[1],)
        self.params['w2']=np.random.rand(self.layers[1],self.layers[2])
        self.params['b2']=np.random.rand(self.layers[2],)

        ''' def initial_weights(self):
        np.random.seed(1)
        self.params['w1']=np.random.randn(self.layers[0],self.layers[1])
        self.params['b1']=np.random.rand(self.layers[1],)
        self.params['w2']=np.random.rand(self.layers[1],self.layers[2])
        self.params['b2']=np.random.rand(self.layers[2],) '''

    def relu(self,z):
        return np.maximum(0,z)
        

    def tanh(self,z):
        z=np.float128(z)
        return 1/(1+np.exp(-z))
    
    
    def back_propagation(self,loss):
        #gradients calculated by loss using backpropagation
        dw1=np.gradient(self.params['w1'],loss,axis=0)
        dw2=np.gradient(self.params['w2'],loss,axis=0)
        db1=np.gradient(self.params['b1'],loss,axis=0)
        db2=np.gradient(self.params['b2'],loss,axis=0)
        dw1=clip_gradients(dw1,1)
        dw2=clip_gradients(dw2,1)
        db1=clip_gradients(db1,1)
        db2=clip_gradients(db2,1) 
        

        #update the weights and bias
        self.params['w1'] = self.params['w1'] - self.lr * dw1
        self.params['w2'] = self.params['w2'] - self.lr * dw2
        self.params['b1'] = self.params['b1'] - self.lr * db1
        self.params['b2'] = self.params['b2'] - self.lr * db2


    def train(self,loss):
        self.loss_store=[]
        #self.initial_weights()
        for i in range(self.iterations):
            self.back_propagation(loss)
            self.loss_store.append(loss)

    def predict(self, X):
        Z1 = X.dot(self.params['w1']) + self.params['b1']
        A1 = self.relu(Z1)
        Z2 = A1.dot(self.params['w2']) + self.params['b2']
        pred = self.relu(Z2)
        return np.round(pred) 

In [ ]:
'''
This is an two layered neural network
when initiating set the number of layers accordingly!!!!!
'''

import numpy as np
from scipy.stats import multivariate_normal
import scipy

class NeuralNet_critic():
    def __init__(self,layers,lr,iterations=1000):
        self.params={}
        self.lr=lr
        self.iterations=iterations
        self.loss=[]
        self.sample_size=None
        self.layers=layers
        np.random.seed(1)
        self.params['w1']=np.random.randn(self.layers[0],self.layers[1])
        self.params['b1']=np.random.rand(self.layers[1],)
        self.params['w2']=np.random.rand(self.layers[1],self.layers[2])
        self.params['b2']=np.random.rand(self.layers[2],)

        ''' def initial_weights(self):
        np.random.seed(1)
        self.params['w1']=np.random.randn(self.layers[0],self.layers[1])
        self.params['b1']=np.random.rand(self.layers[1],)
        self.params['w2']=np.random.rand(self.layers[1],self.layers[2])
        self.params['b2']=np.random.rand(self.layers[2],) '''

    def relu(self,z):
        return np.maximum(0,z)
        

    def tanh(self,z):
        z=np.float128(z)
        return 1/(1+np.exp(-z))
    
    
    def back_propagation(self,loss):
        #gradients calculated by loss using backpropagation
        dw1=np.gradient(self.params['w1'],loss,axis=0)
        dw2=np.gradient(self.params['w2'],loss,axis=0)
        db1=np.gradient(self.params['b1'],loss,axis=0)
        #print(np.shape(self.params['b2']))
        #db2=np.gradient(self.params['b2'],loss,axis=0)
        dw1=clip_gradients(dw1,1)
        dw2=clip_gradients(dw2,1)
        db1=clip_gradients(db1,1) 
        #db2=clip_gradients(db2,1)
        #update the weights and bias
        self.params['w1'] = self.params['w1'] - self.lr * dw1
        self.params['w2'] = self.params['w2'] - self.lr * dw2
        self.params['b1'] = self.params['b1'] - self.lr * db1
        self.params['b2'] = self.params['b2'] - self.lr


    def train(self,loss):
        self.loss_store=[]
        #self.initial_weights()
        for i in range(self.iterations):
            self.back_propagation(loss)
            self.loss_store.append(loss)

    def predict(self, X):
        Z1 = X.dot(self.params['w1']) + self.params['b1']
        A1 = self.relu(Z1)
        Z2 = A1.dot(self.params['w2']) + self.params['b2']
        pred = self.relu(Z2)
        return np.round(pred) 

In [ ]:
'''
This is an implementation of proximal policy optimization with actor and critic neural networks
'''
import numpy as np
#from actornn import NeuralNet_actor
#from crticnn import NeuralNet_critic
from scipy.stats import multivariate_normal
import random




class store:
    def __init__(self):
        self.actions=[]
        self.states=[]
        self.log_probs=[]
        self.rewards=[]
        self.state_values=[]
        self.terminals=[]

    def clear(self):
        del self.actions[:]
        del self.states[:]
        del self.log_probs[:]
        del self.rewards[:]
        del self.state_values[:]
        del self.terminals[:]

class actor_critic():
    def __init__(self,action_dim,states_dim,action_std,lr_actor,lr_critic):
        self.action_dim=action_dim
        self.states_dim=states_dim
        self.action_std=action_std
        self.actor_layers=[self.states_dim,13,self.action_dim]
        self.actor=NeuralNet_actor(self.actor_layers,lr_actor)
        self.critic_layers=[self.states_dim,13,1]
        self.critic=NeuralNet_critic(self.critic_layers,lr_critic)
        p=random.uniform(0.3, 0.5)
        self.cov=np.full((self.action_dim,),p)
        self.cov_mat=np.diag(self.cov)

    def action_pred(self,obs):
        action_mean=self.actor.predict(obs)
        #action=np.random.multivariate_normal(action_mean,self.cov_mat)
        action=np.random.multivariate_normal(action_mean,self.cov_mat)
        log_prob=multivariate_normal.logpdf(action,action_mean,self.cov_mat)
        state_values=self.critic.predict(obs)
        return action,log_prob,state_values
    
    def evaluate(self,obs,action):
        value=self.critic.predict(obs)
        action_mean=self.actor.predict(obs)
        action_mean=np.mean(action_mean,axis=0)
        
        #distribution=multivariate_normal(action_mean,self.cov_mat)
        log_probs=multivariate_normal.logpdf(action,action_mean,self.cov_mat)
        return value,log_probs
    

#implementation of proximal policy optimization

class ppo():
    def __init__(self,states_dim,action_dim,epoch,gamma,lam,clip,lr_actor,lr_critic,action_std):
        self.states_dim=states_dim
        self.actor_dim=action_dim
        self.epoch=epoch
        self.clip=clip
        self.gamma=gamma
        self.lamda=lam
        self.lr_actor=lr_actor
        self.lr_critic=lr_critic
        self.action_std=action_std
        self.local=store()
        self.policy=actor_critic(self.actor_dim,self.states_dim,self.action_std,self.lr_actor,self.lr_critic)
        
    def select_action(self,state):
        action,log_probs,state_values=self.policy.action_pred(state)
        self.local.actions.append(action)
        self.local.log_probs.append(log_probs)
        self.local.states.append(state)
        self.local.state_values.append(state_values)

        return action
    
    def ppo_update(self):
        rewards=[]
        discounted_reward=0
        for reward, terminals in zip(reversed(self.local.rewards), reversed(self.local.terminals)):
            if terminals:
                discounted_reward = 0
            discounted_reward = reward + (self.gamma * discounted_reward)
            rewards.insert(0, discounted_reward)

        rewards=np.array(rewards)
        rewards=(rewards-np.mean(rewards))/(np.std(rewards)+1e7)

        old_states=np.squeeze(np.array(self.local.states))
        old_actions=np.squeeze(np.array(self.local.actions))
        old_state_values=np.squeeze(np.array(self.local.state_values))
        old_log_probs=np.squeeze(np.array(self.local.log_probs))

        advantages=rewards-old_state_values

        for _ in range(self.epoch):
            state_values,log_probs=self.policy.evaluate(old_states,old_actions)

            log_probs=np.array(log_probs)

            #ratio
            ratio=np.exp(log_probs-old_log_probs)
            surr1 = ratio* advantages
            surr2 = np.clip(ratio, 1-self.clip, 1+self.clip) * advantages
            
            #actor loss
            actor_loss=-np.mean(np.minimum(surr1,surr2))
            print("a",actor_loss)
            #print("aaa",np.shape(actor_loss))
            state_values=np.array(state_values)
            #critic loss
            critic_loss=np.mean(np.square(state_values-rewards))
            print("aa",critic_loss)
            #print("aa",critic_loss)
            #updating the weights of actor neural net
            self.policy.actor.train(actor_loss)

            #updating the weights of critic neural net
            self.policy.critic.train(critic_loss)

        self.local.clear()

    
    


In [ ]:
'''
This code is for training the policy
Considering only continuous action spaces
'''

import numpy as np
import os
import glob
import time
from datetime import datetime
import gym
import csv
#from ppo import ppo
#from ppo import policy
'''
for the first training we will be using cartpole_v1
'''
def train():
    #intitiating the environment parameters for training
    env_name="BipedalWalker-v3"

    #max timesteps in an episode
    max_episode_len=1500
    #max timesteps in one training
    max_training_time=int(4e6)

    #printing reward frequency
    frequency_print=max_episode_len*10
    frequency_log=max_episode_len*2

    #action distribution particularly multivariate normal are 
    #being used for finding the probability of an action in states
    action_dist=0.6

    #below are the parameters for proximal policy optimization
    update_policy_step=max_episode_len*4
    #here the policy is being updated one time for epoch times
    epoch=1
    #clip is the ratio of new_policy upon the old_policy 
    clip=0.2
    #gamma is the discount factor
    gamma=0.99 
    lam=0.95
    #learning rates for actor and critic
    lr_actor=0.0004
    lr_critic=0.03

    #random seed for starting action
    random_seed=1
    #parameters end here
    print("Initialising the continuous action state policy")
    print("training the policy in : " +env_name)
    #inittaiting the gym
    env=gym.make(env_name)

    #getting the dimensions of observation of agent in environment
    state_dim=env.observation_space.shape[0]

    #getting the dimensions of action space of agent
    #action_dim=env.action_space.shape[0]
    #use above command if the action dim is greater than 1
    action_dim=env.action_space.shape[0]

    #now logging the policy and rewards in csv files

    log_dir="ppo_logs"
    if not os.path.exists(log_dir):
        os.makedirs(log_dir)

    log_dir=log_dir+'/'+env_name+'/'
    if not os.path.exists(log_dir):
        os.makedirs(log_dir)

    #look for this command online
    num=0
    current_num=next(os.walk(log_dir))[2]
    num=len(current_num)

    #creating the logger file for each lenght of episode
    logger_name=env_name+str(num)+".csv"
    print("current logger number for " +env_name+":",num)

    #starting the original training
    run_number=0 #we need to comstantly change this number as we train 
    #saving the data
    

    #printing all the data
    print("maximum training steps:",max_training_time)
    print("maximum episode time:",max_episode_len)
    print("frequency of model saving:",model_save_freq)
    print("logging frequency:",frequency_log)
    print("showing reward frequency:",frequency_print)
    print("action dim:",action_dim)
    print("observations dim:",state_dim)
    print("action distribution or multivariate normal:",action_dist)
    print("decay rate of action distribution:",action_dist_decay_rate)
    print("decay frequency of action distibution:",action_dist_decay_freq)
    print("minimum action distribution:",min_action_dist)
    print("policy update frequency:",update_policy_step)
    print("epoch:",epoch)
    print("policy ratio clip:",clip)
    print("discount factor:",gamma)
    print("actor learning rate:",lr_actor)
    print("critic learning rate:",lr_critic)
    if random_seed:
        print("random seed",random_seed)
        np.random.seed(random_seed)

    print("Proximal policy initialisation begins!")
    #here we will initialise the policy of agent
    ppo_agent=ppo(state_dim,action_dim,epoch,gamma,0.95,0.2,lr_actor,lr_critic,action_dist)
    #begin time
    start_time=datetime.now().replace(microsecond=0)
    print("starting the training time at:",start_time)
    log=open(logger_name,"w+")
    log.write('episode,time,reward\n')

    present_reward=0
    present_episode=0
    run_log_reward=0
    run_log_episodes=0

    time_step=0
    i_episode=0
    
    #training begins
    while time_step<max_training_time:
        state=env.reset()
        #it=np.array(state)
        #bit=it.reshape(96,-1)
        #print(np.shape(state))
        #print(np.shape(bit))
        current_episode_reward=0
        for t in range(1,max_episode_len+1):
            #here the action is being selected using the policy
            action=ppo_agent.select_action(state)
            #print("aaa",action)
            state,reward,done,_=env.step(action)
            
            #saving the rewards and terminals
            #ppo_agent.store.rewards.append(reward)
            #ppo_agent.store.terminals.append(done)
            ppo_agent.local.rewards.append(reward)
            ppo_agent.local.terminals.append(done)

            time_step+=1
            current_episode_reward+=reward

            #update the ppo agent
            #test15 updating the ppo every time
            if time_step % frequency_print ==0:
                present_reward_avg=present_reward/present_episode
                present_reward_avg=round(present_reward_avg,2)

                print("Episode:{} \t Timestep:{} \t Average Reward:{} \t".format(i_episode,time_step,present_reward_avg))
                present_reward=0
                present_episode=0


            if time_step %update_policy_step==0:
                ppo_agent.ppo_update()

                

            if time_step % frequency_log==0:
                avg_log_reward=run_log_reward/run_log_episodes
                avg_log_reward=round(avg_log_reward,4)

                log.write('{},{},{}\n'.format(i_episode,time_step,avg_log_reward))
                log.flush()

                run_log_reward=0
                run_log_episodes=0

            

            

            #ppo_agent.delete()

            if done:
                break

        present_reward+=current_episode_reward
        present_episode+=1
        run_log_reward+=current_episode_reward
        run_log_episodes+=1

        i_episode+=1

    log.close()
    env.close

    print("-----------------------------------------------------------------")
    end_time=datetime.now().replace(microsecond=0)
    print("started training at",start_time)
    print("ended training at:",end_time)
    print("total training time:",end_time-start_time)
    print("-----------------------------------------------------------------")
    


if __name__=='__main__':
    train()

-----------------------------------------------------------------
Initialising the continuous action state policy
training the policy in : BipedalWalker-v3
current logger number for BipedalWalker-v3: 0
-----------------------------------------------------------------
maximum training steps: 3000000
maximum episode time: 1000
frequency of model saving: 100000
logging frequency: 2000
showing reward frequency: 10000
action dim: 4
observations dim: 24
action distribution or multivariate normal: 0.6
decay rate of action distribution: 0.02
decay frequency of action distibution: 250000
minimum action distribution: 0.02
policy update frequency: 4000
epoch: 1
policy ratio clip: 0.2
discount factor: 0.99
actor learning rate: 0.0003
critic learning rate: 0.002
random seed 1
-----------------------------------------------------------------
Proximal policy initialisation begins!
starting the training time at: 2023-02-17 09:54:01
a 22.799068083709507
aa 247.15925000000536
a 15.876411169414713
aa 188

/usr/local/lib/python3.8/dist-packages/numpy/lib/function_base.py:1080: RuntimeWarning: overflow encountered in true_divide
  out[tuple(slice1)] = (f[tuple(slice4)] - f[tuple(slice2)]) / (2. * ax_dx)
<ipython-input-27-5570ce58ad9e>:12: RuntimeWarning: invalid value encountered in multiply
  clipped_grads = grads * (clip_threshold / grad_norm)
/usr/local/lib/python3.8/dist-packages/numpy/lib/function_base.py:1101: RuntimeWarning: overflow encountered in true_divide
  out[tuple(slice1)] = (f[tuple(slice2)] - f[tuple(slice3)]) / dx_0
/usr/local/lib/python3.8/dist-packages/numpy/lib/function_base.py:1108: RuntimeWarning: overflow encountered in true_divide
  out[tuple(slice1)] = (f[tuple(slice2)] - f[tuple(slice3)]) / dx_n
/usr/local/lib/python3.8/dist-packages/numpy/lib/function_base.py:1101: RuntimeWarning: overflow encountered in double_scalars
  out[tuple(slice1)] = (f[tuple(slice2)] - f[tuple(slice3)]) / dx_0
/usr/local/lib/python3.8/dist-packages/numpy/lib/function_base.py:1108: Runt

AssertionError: ignored